In [ ]:
import pickle
import os
import random
import sqlite3
import string,numpy

In [ ]:
class ReviewEmbedding:
    def review_emb(self,text, word_emb,word_dict):
        words = []
        rev_w = text.lower().strip().split()
        for w in rev_w:
            prev = 0
            for ch in range(len(w)):
                if w[ch] in self.punc:
                    if prev < ch:
                        words += [w[prev:ch]]
                    words += [w[ch]]
                    prev = ch + 1
            if prev < len(w):
                words += [w[prev:]]
        emb = numpy.array([0.0]*64)
        for word in words:
            if word in word_dict:
                emb += word_emb[word_dict[word]]
            else:
                word = '<UNK>'
                emb += word_emb[word_dict[word]]
        return emb
    
    def Select(self,q):
        return self.c.execute(q)

    def Update(self,q):
        self.c.execute(q)
        self.conn.commit()
    
    def __init__(self):
        self.review_dict = {}
        self.c = None
        self.conn = None
        self.review_dict = None
        self.punc = set(string.punctuation)
    
    def create(self,db_path,word_dict_path, word_emb_path,review_delim_path,review_emb_path):
        
        self.conn = sqlite3.connect(db_path)
        self.c = self.conn.cursor()
        q = self.Select("select user_id,business_id, starts, review_id, review_text from Reviews ")
        word_dict = pickle.load(open(word_dict_path,"rb"))
        word_emb = pickle.load(open(word_emb_path,"rb"))
        review_dict = {}
        i=0
        review_dict = {}
        prev = 0
        with open(review_emb_path,"w") as f:
            for row in q:
                user_id=row[0]
                business_id=row[1]
                rating=row[2]
                review_id = row[3]
                emb = self.review_emb(row[4],word_emb,word_dict)
                line = user_id+"\t"+business_id+"\t"+rating+"\t"+review_id+"\t"+ " ".join([str(x) for x in emb])
                f.write(line+"\n")
                
                review_dict[review_id] = (prev,len(line)+1)
                prev += len(line)+1
                if i%100000==0:
                    print i," "
                i+=1
        pickle.dump(review_dict,open(review_delim_path,"wb"))
        self.conn.close()
    
  

In [75]:
re = ReviewEmbedding()

In [ ]:
re.create("yelp_dataset.db","dict.pkl","emb.pkl","review_delim.pkl","review_emb.txt")